In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

/Users/liuguanhong/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# URLs to scrape for 2020 and 2021 survey data
urls = [
    "https://www.census.gov/programs-surveys/household-pulse-survey/data/tables.2020.html#list-tab-404305343",
    "https://www.census.gov/programs-surveys/household-pulse-survey/data/tables.2021.html#list-tab-404305343"
]

# List to store the results from both pages
survey_links = []

# Function to scrape the survey links from the given URL
def scrape_survey_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    container = soup.find_all('a', class_='uscb-list-item')
    
    # Extract the titles and URLs
    for item in container:
        title = item.find('div', class_='uscb-default-x-column-title').get_text(strip=True)
        link = "https://www.census.gov" + item['href']
        survey_links.append({"title": title, "link": link})

# Scrape both 2020 and 2021 pages
for url in urls:
    scrape_survey_links(url)

# Convert to a pandas df for better display
df = pd.DataFrame(survey_links)

df

,title,link
0,Week 20 Household Pulse Survey: November 25 – ...,https://www.census.gov/data/tables/2020/demo/h...
1,Week 19 Household Pulse Survey: November 11 – ...,https://www.census.gov/data/tables/2020/demo/h...
2,Week 18 Household Pulse Survey: October 28 – N...,https://www.census.gov/data/tables/2020/demo/h...
3,Week 17 Household Pulse Survey: October 14 – O...,https://www.census.gov/data/tables/2020/demo/h...
4,Week 16 Household Pulse Survey: September 30 –...,https://www.census.gov/data/tables/2020/demo/h...
5,Week 15 Household Pulse Survey: September 16 –...,https://www.census.gov/data/tables/2020/demo/h...
6,Week 14 Household Pulse Survey: September 2 – ...,https://www.census.gov/data/tables/2020/demo/h...
7,Week 13 Household Pulse Survey: August 19 – Au...,https://www.census.gov/data/tables/2020/demo/h...
8,Week 12 Household Pulse Survey: July 16 - Jul...,https://www.census.gov/data/tables/2020/demo/h...
9,Week 11 Household Pulse Survey: July 9 - July 14,https://www.census.gov/data/tables/2020/demo/h...


In [3]:
# Extract the week number from the 'Title'
df['week'] = df['title'].apply(lambda x: re.search(r'Week (\d+)', x).group(1) if re.search(r'Week (\d+)', x) else None)

# Extract the time period from the 'Title'
df['month'] = df['title'].apply(lambda x: re.search(r':\s*(.*)', x).group(1) if re.search(r':\s*(.*)', x) else None)

# Extract the year from the 'Link' and append it to 'Time Period'
df['month'] = df['month'] + " (" + df['link'].apply(lambda x: re.search(r'tables/(\d{4})', x).group(1) if re.search(r'tables/(\d{4})', x) else None) + ")"

# Drop rows where 'Week' is None before converting 'Week' to int
df = df.dropna(subset=['week']).copy()

# Convert Week column to integers safely without triggering the warning
df['week'] = df['week'].astype(int)

# Filter for Week 1 to Week 33 and sort by 'Week'
filtered_df = df[(df['week'] >= 1) & (df['week'] <= 33)].copy().sort_values(by='week')

filtered_df

,title,link,week,month
19,Week 1 Household Pulse Survey: April 23 - May 5,https://www.census.gov/data/tables/2020/demo/h...,1,April 23 - May 5 (2020)
18,Week 2 Household Pulse Survey: May 7 - May 12,https://www.census.gov/data/tables/2020/demo/h...,2,May 7 - May 12 (2020)
17,Week 3 Household Pulse Survey: May 14 - May 19,https://www.census.gov/data/tables/2020/demo/h...,3,May 14 - May 19 (2020)
16,Week 4 Household Pulse Survey: May 21 - May 26,https://www.census.gov/data/tables/2020/demo/h...,4,May 21 - May 26 (2020)
15,Week 5 Household Pulse Survey: May 28 - June 2,https://www.census.gov/data/tables/2020/demo/h...,5,May 28 - June 2 (2020)
14,Week 6 Household Pulse Survey: June 4 - June 9,https://www.census.gov/data/tables/2020/demo/h...,6,June 4 - June 9 (2020)
13,Week 7 Household Pulse Survey: June 11 - June 16,https://www.census.gov/data/tables/2020/demo/h...,7,June 11 - June 16 (2020)
12,Week 8 Household Pulse Survey: June 18 - June 23,https://www.census.gov/data/tables/2020/demo/h...,8,June 18 - June 23 (2020)
11,Week 9 Household Pulse Survey: June 25 - June 30,https://www.census.gov/data/tables/2020/demo/h...,9,June 25 - June 30 (2020)
10,Week 10 Household Pulse Survey: July 2 - July 7,https://www.census.gov/data/tables/2020/demo/h...,10,July 2 - July 7 (2020)


In [4]:
# Path to the folder containing all Excel files
folder_path = './survey data'

# List to store the extracted data
data_list = []

# Loop through all the Excel files in the folder
for i in range(1, 34):  # Loop through health1_week1.xlsx to health1_week33.xlsx
    file_name = f'health1_week{i}.xlsx'
    file_path = os.path.join(folder_path, file_name)
    
    # Read the Excel file, skip the first rows to check if data starts further down
    xl = pd.ExcelFile(file_path)
    
    # Loop through all the sheets (AL to WY) and extract the needed data
    for sheet_name in xl.sheet_names:
        if 'US' in sheet_name or 'Metro_Area' in sheet_name:
            continue  # Skip sheets with 'US' or 'Metro_Area' in the name
        
        df = xl.parse(sheet_name, skiprows=4)  # Skip 4 rows to adjust header
        
        # Clean the column names
        df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
        df.columns = df.columns.str.replace('\n', '')  # Remove newlines

        # Try to find the 'Total' row and corresponding columns for Yes/No data
        try:
            total_row = df[df.iloc[:, 0].str.contains('Total', na=False)]  # Locate the 'Total' row
            
            # Now identify the correct columns based on known structure (Yes/No for delayed and needed)
            delayed_total_yes = total_row.iloc[0, 1]  # Yes is in the second column
            delayed_total_no = total_row.iloc[0, 2]  # No is in the third column
            
            # Append the extracted data to the list
            data_list.append({
                'week': i,
                'state': sheet_name,
                'delayed total yes': delayed_total_yes,
                'delayed total no': delayed_total_no
            })
        except Exception as e:
            print(f"Error processing sheet {sheet_name} in file {file_name}: {e}")

# Convert the data list to a df
df_extracted = pd.DataFrame(data_list)
df_extracted

,week,state,delayed total yes,delayed total no
0,1,AL,1206233,1998924
1,1,AK,203137,273453
2,1,AZ,1818427,3167005
3,1,AR,788832,1119152
4,1,CA,10639086,16315830
...,...,...,...,...
1678,33,VA,695825,4163951
1679,33,WA,794253,3847327
1680,33,WV,140156,871777
1681,33,WI,284528,3057451


In [5]:
# Calculate the ratio: Delayed Total Yes / (Delayed Total Yes + Delayed Total No)
df_extracted['delayed ratio'] = df_extracted['delayed total yes'] / (df_extracted['delayed total yes'] + df_extracted['delayed total no'])
df_extracted

,week,state,delayed total yes,delayed total no,delayed ratio
0,1,AL,1206233,1998924,0.376341
1,1,AK,203137,273453,0.426230
2,1,AZ,1818427,3167005,0.364748
3,1,AR,788832,1119152,0.413437
4,1,CA,10639086,16315830,0.394699
...,...,...,...,...,...
1678,33,VA,695825,4163951,0.143180
1679,33,WA,794253,3847327,0.171117
1680,33,WV,140156,871777,0.138503
1681,33,WI,284528,3057451,0.085138


In [6]:
# Merging df_extracted with filtered_df based on the 'Week' column to add 'Time Period' to df_extracted
df_extracted = pd.merge(df_extracted, filtered_df[['week', 'month']], on='week', how='left')

df_extracted

,week,state,delayed total yes,delayed total no,delayed ratio,month
0,1,AL,1206233,1998924,0.376341,April 23 - May 5 (2020)
1,1,AK,203137,273453,0.426230,April 23 - May 5 (2020)
2,1,AZ,1818427,3167005,0.364748,April 23 - May 5 (2020)
3,1,AR,788832,1119152,0.413437,April 23 - May 5 (2020)
4,1,CA,10639086,16315830,0.394699,April 23 - May 5 (2020)
...,...,...,...,...,...,...
1678,33,VA,695825,4163951,0.143180,June 23 – July 5 (2021)
1679,33,WA,794253,3847327,0.171117,June 23 – July 5 (2021)
1680,33,WV,140156,871777,0.138503,June 23 – July 5 (2021)
1681,33,WI,284528,3057451,0.085138,June 23 – July 5 (2021)


In [7]:
# Function to convert the month format and only keep the end date (after "to")
def extract_end_date(date_range_str):
    # Replace en dash with regular hyphen
    date_range_str = date_range_str.replace("–", "-")
    
    # Extract the month and day parts using regex
    match = re.match(r"(\w+)\s(\d+)\s-\s(\w+)\s(\d+)\s\((\d+)\)", date_range_str)
    if match:
        start_month, start_day, end_month, end_day, year = match.groups()

        # Convert the month names to numbers
        months = {
            "January": "01", "February": "02", "March": "03", "April": "04",
            "May": "05", "June": "06", "July": "07", "August": "08",
            "September": "09", "October": "10", "November": "11", "December": "12"
        }

        end_month_num = months.get(end_month)

        if end_month_num:
            # Return only the end date
            return f"{year}-{end_month_num}-{int(end_day):02d}"
    
    # If the format is unexpected, return the original string
    return date_range_str

# Apply the function to the 'month' column
df_extracted['month'] = df_extracted['month'].apply(extract_end_date)

# Check if any rows have not been converted correctly
problematic_rows = df_extracted[~df_extracted['month'].str.contains(r'\d{4}-\d{2}-\d{2}', na=False)]

# Display the problematic rows if any
if not problematic_rows.empty:
    print("Some rows were not converted correctly:")
    print(problematic_rows)
else:
    print("All rows have been successfully converted to the end date format 'YYYY-MM-DD'.")

# Display the updated dataframe
df_extracted

All rows have been successfully converted to the end date format 'YYYY-MM-DD'.


,week,state,delayed total yes,delayed total no,delayed ratio,month
0,1,AL,1206233,1998924,0.376341,2020-05-05
1,1,AK,203137,273453,0.426230,2020-05-05
2,1,AZ,1818427,3167005,0.364748,2020-05-05
3,1,AR,788832,1119152,0.413437,2020-05-05
4,1,CA,10639086,16315830,0.394699,2020-05-05
...,...,...,...,...,...,...
1678,33,VA,695825,4163951,0.143180,2021-07-05
1679,33,WA,794253,3847327,0.171117,2021-07-05
1680,33,WV,140156,871777,0.138503,2021-07-05
1681,33,WI,284528,3057451,0.085138,2021-07-05


In [8]:
# Define a function to subtract 4 weeks (28 days) from the date
def subtract_4_weeks(date_str):
    try:
        # Convert the date string to a datetime object
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        # Subtract 28 days
        new_date = date_obj - timedelta(weeks=4)
        # Return the new date as a string
        return new_date.strftime('%Y-%m-%d')
    except ValueError:
        # If there's an issue converting, return the original string
        return date_str

# Create a new df and subtract 4 weeks from the 'month' column
df_extracted_minus_4 = df_extracted.copy()
df_extracted_minus_4['month'] = df_extracted_minus_4['month'].apply(subtract_4_weeks)

# Display the updated df
df_extracted_minus_4

,week,state,delayed total yes,delayed total no,delayed ratio,month
0,1,AL,1206233,1998924,0.376341,2020-04-07
1,1,AK,203137,273453,0.426230,2020-04-07
2,1,AZ,1818427,3167005,0.364748,2020-04-07
3,1,AR,788832,1119152,0.413437,2020-04-07
4,1,CA,10639086,16315830,0.394699,2020-04-07
...,...,...,...,...,...,...
1678,33,VA,695825,4163951,0.143180,2021-06-07
1679,33,WA,794253,3847327,0.171117,2021-06-07
1680,33,WV,140156,871777,0.138503,2021-06-07
1681,33,WI,284528,3057451,0.085138,2021-06-07


In [9]:
# Modify the code to filter the data for the period between 2020-04 and 2021-05

# Define the function to extract year-month from 'month'
def extract_year_month(date_str):
    try:
        # Convert to datetime object and extract year-month
        return datetime.strptime(date_str, '%Y-%m-%d').strftime('%Y-%m')
    except ValueError:
        return date_str

# Apply the function to extract 'month' as year-month in df_extracted_minus_4
df_extracted_minus_4['month'] = df_extracted_minus_4['month'].apply(extract_year_month)

# Filter the data for the date range 2020-04 to 2021-05
df_filtered = df_extracted_minus_4[(df_extracted_minus_4['month'] >= '2020-04') & (df_extracted_minus_4['month'] <= '2021-05')]

# Calculate the weighted average of 'delayed ratio' by grouping by 'state' and 'month'
df_weighted_avg_minus_4 = df_filtered.groupby(['state', 'month']).apply(
    lambda x: (x['delayed ratio'] * x['week']).sum() / x['week'].sum()
).reset_index(name='weighted_delayed_ratio')

df_weighted_avg_minus_4

,state,month,weighted_delayed_ratio
0,AK,2020-04,0.453173
1,AK,2020-05,0.471273
2,AK,2020-06,0.452521
3,AK,2020-08,0.370384
4,AK,2020-09,0.355726
...,...,...,...
658,WY,2021-01,0.238784
659,WY,2021-02,0.214587
660,WY,2021-03,0.203182
661,WY,2021-04,0.108362


In [10]:
# Loading the sentiment_validation.csv dataset from the provided path
sentiment_df = pd.read_csv('./sentiment_validation.csv')

# Display the first few rows to ensure it loaded correctly
sentiment_df

,state,month,review_count,total_sentiment,average_sentiment
0,Alabama,2020-04,50,-10,-0.200000
1,Alabama,2020-05,173,-19,-0.109827
2,Alabama,2020-06,209,-26,-0.124402
3,Alabama,2020-07,457,-35,-0.076586
4,Alabama,2020-08,372,-8,-0.021505
...,...,...,...,...,...
695,Wyoming,2020-12,39,0,0.000000
696,Wyoming,2021-01,34,-4,-0.117647
697,Wyoming,2021-02,22,-1,-0.045455
698,Wyoming,2021-03,30,3,0.100000


In [11]:
# State abbreviation to full name mapping
state_abbr_to_name = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Map state abbreviations to full names in df_weighted_avg_minus_4
df_weighted_avg_minus_4['state'] = df_weighted_avg_minus_4['state'].map(state_abbr_to_name)

# Merge sentiment_df and df_weighted_avg_minus_4 based on 'state' and 'month'
merged_df = pd.merge(sentiment_df, df_weighted_avg_minus_4, on=['state', 'month'], how='inner')

# Calculate the correlation between 'average_sentiment' and 'weighted_delayed_ratio'
correlation_value = merged_df['average_sentiment'].corr(merged_df['weighted_delayed_ratio'])

# Display the correlation value
correlation_value

-0.22379529423933484